In [ ]:
pip install numpy pandas scikit-learn nltk imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk .tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")

In [ ]:

df = pd.read_csv(r"C:\Users\Shreyam Sharma\Desktop\dataset\first_aid_data.csv", skiprows=1, names=["Emergency Condition", "Symptoms", "First Aid Steps"])


df.dropna(inplace=True)


print(df.head())

In [ ]:

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', "", text) 
 


    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    words= words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words("english")]
    return ' '.join(words)

df['Symptoms'] = df['Symptoms'].apply(preprocess_text)
df["Emergency Condition"] = df["Emergency Condition"].str.strip().str.lower()


df




In [ ]:
print(df.columns)

In [ ]:
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1,2))
x = vectorizer.fit_transform(df["Symptoms"])
y = df["Emergency Condition"]


In [ ]:
from sklearn.model_selection import StratifiedKFold





skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)





for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

 
    svm_model = SVC(kernel='linear', C=1, random_state=42)
    svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)

   
    print(classification_report(y_test, y_pred))

In [ ]:
def predict_emergency(symptoms_text):
    symptoms_text = preprocess_text(symptoms_text)
    input_vector= vectorizer.transform([symptoms_text])
    prediction = svm_model.predict(input_vector)
    first_aid = df[df['Emergency Condition']==prediction[0]]['First Aid Steps'].values[0]
    return prediction[0], first_aid



In [ ]:
import ipywidgets as widgets
from IPython.display import display


symptoms_input = widgets.Text(description="Symptoms:")
submit_button = widgets.Button(description="Submit")


output = widgets.Output()


display(symptoms_input, submit_button, output)

def on_submit(_):
    with output:
        output.clear_output()  
        symptoms_text = symptoms_input.value.strip()

        if not symptoms_text:
            print("Please enter symptoms before submitting.")
            return

        if symptoms_text.lower() == 'exit':
            print("Exiting program...")
            return

        try:
            predicted_condition, first_aid_steps = predict_emergency(symptoms_text)
            print(f"\nPredicted Emergency Condition: {predicted_condition}")
            print(f"Recommended First Aid Steps: {first_aid_steps}\n")
        except IndexError:
            print("No matching medical condition found. Please try different symptoms.\n")


submit_button.on_click(on_submit)
